In [5]:
import numpy as np
import flet as ft
import scipy.integrate as integrate
from sympy import sympify, lambdify
from sympy.abc import x
import nest_asyncio
nest_asyncio.apply()

In [6]:
def string_to_function(expression):
    if 'x' not in expression:
        # here we say that the expression in input has to contain the variable x
        raise SyntaxError("Error: The function must include the variable 'x'. Example: 'exp(-x^2/2)'")
    try:
        sympy_expr = sympify(expression) # with the sympify() function we convert the string into a symbolic mathematical expression

        f = lambdify(x, sympy_expr, modules=["numpy"]) # with lambdify() we convert the mathematical expression into a python expression
    except Exception:
        raise SyntaxError('Error: Please enter a correct function, as exp(-x^2/2)')
    return f # we convert it to a Python function

def read_range(range_text):
    # we check to see if the range text is empty or not
    if not range_text:
        raise AttributeError('Error: Please enter a range')
    nums = range_text.split(",")

    # the range is defined by two numbers separated by a comma, this is what we check below
    if len(nums) != 2:
        raise TypeError('Error: Invalid range. Use two integer values separated by a comma')
    
    try:
        # convert both values to floats
        a, b = float(nums[0].strip()), float(nums[1].strip())
    except ValueError:
        # in case someone tries to insert a string or something else insead of a float number as a range
        raise TypeError('Error: Invalid range. Use two numerical values separated by a comma')
    
    if a >= b:
        raise ValueError('Error: The first number must be lower than the second')

    return a, b

def check_function_defined_nonnegative(pdf_function, a, b):
    # Check if the function is well-defined within the range [a, b]
    x_values = np.linspace(a, b, 1000)
    y_values = pdf_function(x_values)
    if np.any(np.isnan(y_values)) or np.any(np.isinf(y_values)):
        raise ValueError("The function is not defined in the given range.")
    
    # we also check that the function is non-negative in the given range, so it can be considered as a valid PDF.
    if np.any(y_values < 0):
        raise ValueError("The function is negative in the given range, so it cannot be a valid PDF.")

# function that we can use tu normalize our pdf
def normalize_pdf(pdf_function, a, b):
    check_function_defined_nonnegative(pdf_function, a, b)
    area, _ = integrate.quad(pdf_function, a, b)
    # area can never be negative
    if area <= 0:
        raise ValueError("PDF cannot be normalized")
    return lambda x: pdf_function(x) / area

def main(page: ft.Page):
    page.title = 'Exercise 2 - Part B'
    page.window.width = 750
    page.window.height = 500
    page.scroll = ft.ScrollMode.ALWAYS

    # the lloyd_algorithm() function is called when we run the code, and also when we change the input value and click the run button
    def lloyd_algorithm(e = None):
        # Initialize display elements
        distortion_text.value = ""
        quantization_table.rows = []
        pdf = pdf_input.value
        ran = range_input.value
        num = quantization_points_input.value
        # we don't go on with the function if the PDF of X field or the range field or the number of quantization points field is empty
        if not pdf or not ran or not num:
            return
        
        try:
            # Read and parse the range
            a, b = read_range(ran)
            # Convert the input PDF expression into a usable function
            pdf_function = string_to_function(pdf)
            n = int(num)

            # Normalize the PDF function and check whether all values ​​are non-negative within the specified range
            normalized_pdf = normalize_pdf(pdf_function, a, b)
            check_function_defined_nonnegative(normalized_pdf, a, b)

            # Generate initial quantization points
            quant_points = np.linspace(a, b, n)

            # Perform up to 100 iterations to optimize the quantization points
            for _ in range(100):
                # Calculate the threshold between quantization points
                thresholds = (quant_points[:-1] + quant_points[1:]) / 2
                new_quant_points = []

                # Calculate new quantization points for each interval
                for j in range(len(thresholds) + 1):
                    if j == 0:
                        lower = a
                    else:
                        lower = thresholds[j - 1]
                    if j == len(thresholds):
                        upper = b
                    else:
                        upper = thresholds[j]
                    # Calculate the new quantization point position
                    integral_xf = integrate.quad(lambda x: x * normalized_pdf(x), lower, upper)[0]
                    integral_f = integrate.quad(normalized_pdf, lower, upper)[0]
                    new_quant_points.append(integral_xf / integral_f)
                quant_points = np.array(new_quant_points)

            # Clear previous quantization points in the quantization table
            quantization_table.rows.clear()

            # Add new quantization points in the quantization table
            for point in quant_points:
                quantization_table.rows.append(ft.DataRow(cells=[ft.DataCell(ft.Text(f"{point:.6f}"))]))

            # Compute Optimum distortion
            t = [-float('inf')]
            for x in thresholds:
                t.append(x)
            t.append(float('inf'))
            distortion = sum([integrate.quad(lambda x: (x - quant_points[i])**2 * normalized_pdf(x), t[i], t[i + 1])[0] for i in range(len(quant_points))])
            distortion_text.value = f"{distortion:.6f}"

            # Clear errors content
            errors.value = ""
        except Exception as ex:
            # Add errors if occur
            errors.value = str(ex)

        page.update()

    # UI Setup
    page.add(ft.Text('Lloyd algorithm', width=750, height=50, text_align=ft.TextAlign.CENTER, weight=ft.FontWeight.BOLD, size=30))

    # when we write something into the textfield and then click the run button, the lloyd_algorithm function is called
    pdf_input = ft.TextField(width=200, value="exp(-x^2/2)")
    range_input = ft.TextField(width=200, value="-10,10")
    quantization_points_input = ft.TextField(width=200, value="2")
    distortion_text = ft.TextField(width=200, read_only=True)
    run_button = ft.ElevatedButton("RUN", on_click=lambda e: lloyd_algorithm())
    exit_button = ft.ElevatedButton("EXIT", on_click=lambda e: page.window.close())

    # Input Column
    r1 = ft.Row(controls=[ft.Text('PDF of X (to be normalized)'), pdf_input], height=50, alignment=ft.MainAxisAlignment.END)
    r2 = ft.Row(controls=[ft.Text('Range of x (a,b)'), range_input], height=50, alignment=ft.MainAxisAlignment.END)
    r3 = ft.Row(controls=[ft.Text('Number of quantization points'), quantization_points_input], height=50, alignment=ft.MainAxisAlignment.END)
    r4 = ft.Row(controls=[ft.Text('Optimum distortion'), distortion_text], height=50, alignment=ft.MainAxisAlignment.END)
    r5 = ft.Row(controls=[run_button, exit_button], height=50, alignment=ft.MainAxisAlignment.END)
    input_column = ft.Column(controls=[r1, r2, r3, r4, r5], horizontal_alignment=ft.CrossAxisAlignment.START, spacing=10)

    # Quantization points DataTable
    quantization_table = ft.DataTable(
        columns=[
            ft.DataColumn(ft.Text("Quantization points"), numeric=True)
        ],
        rows=[], # Empty initially; rows will be added after computation
        width=200,
        border=ft.border.all(1, ft.colors.BLACK),
        horizontal_lines=ft.BorderSide(1, "black")
    )

    main_row = ft.Row(
        controls=[
            ft.Container(content=input_column, expand=4),
            ft.Container(content=quantization_table, expand=2)
        ]
    )

    page.add(main_row)
    errors = ft.Text(color="red") # Error message area
    page.add(errors)

    lloyd_algorithm()

    page.update()

In [ ]:
# Run the app
# Work only on Windows 11 and macOS. If using Windows, comment this line and use the web browser option below.
ft.app(target=main, view=ft.AppView.FLET_APP)

In [ ]:
# Works on all operating systems, including Windows 10. Uncomment this line if using Windows 10 and comment the pop-up window option.
#ft.app(target=main, view=ft.AppView.WEB_BROWSER)